In [1]:
using LinearAlgebra
using Optim
using HDF5

Fidelity for qubit can be written:

$F(\rho, \sigma) = \operatorname{tr}(\rho \sigma) + 2\sqrt{\det(\rho) \det(\sigma)}$

In [2]:
function infidelity_norm(ρ, σ)
    1 - tr(ρ * σ) - 2*sqrt(det(ρ)*det(σ))
end

infidelity_norm (generic function with 1 method)

In [7]:
function read_timeevolution(file_name, state, γ)
    h5open(file_name, "r") do file
        ρᵧ = read(file[state][string(γ)])
        t = ρᵧ["t"]
        ρ₀₀ = ρᵧ["p0"]; Re_ρ₀₁ = ρᵧ["s_re"];  Im_ρ₀₁ = ρᵧ["s_im"]
        ρ_series = []
        t_series = []

        for i in 1:length(t)
            ρᵢ= [ ρ₀₀[i]                      Re_ρ₀₁[i] + im * Im_ρ₀₁[i]
                  Re_ρ₀₁[i] - im * Im_ρ₀₁[i]  1 - ρ₀₀[i]                 ]
            push!(ρ_series, convert(Matrix{ComplexF64}, ρᵢ))
            push!(t_series, convert(Float64, t[i]))
        end
        return(t_series, ρ_series)
    end
end

# Define the Pauli matrices
σˣ = [0 1; 1 0]
σʸ = [0 im; -im 0]
σᶻ = [1 0; 0 -1]

# Define the basis elements
fᴷ₁ = σˣ / 2
fᴷ₂ = σʸ / 2
fᴷ₃ = σᶻ / 2

# Check orthogonality and normalization
@assert tr(fᴷ₁ * fᴷ₂) ≈ 0
@assert tr(fᴷ₁ * fᴷ₃) ≈ 0
@assert tr(fᴷ₂ * fᴷ₃) ≈ 0
@assert tr(fᴷ₁ * fᴷ₁) ≈ 1 / 2
@assert tr(fᴷ₂ * fᴷ₂) ≈ 1 / 2
@assert tr(fᴷ₃ * fᴷ₃) ≈ 1 / 2

fᴼᴺᴮ = [fᴷ₁, fᴷ₂, fᴷ₃]

# Function to calculate Dc
function Dc(ρ, t, H, C)
    U = (H * ρ - ρ * H) / im
    D = sum(C .* [2 * fᵢ * ρ * fⱼ' - ρ * fⱼ' * fᵢ - fⱼ' * fᵢ * ρ for fᵢ in fᴼᴺᴮ, fⱼ in fᴼᴺᴮ]) / 2
    return U + D
end

function construct_H(params)
    ϵ, h_Re, h_Im = params[1:3]
    return [
        ϵ               h_Re + im * h_Im
        h_Re - im * h_Im -ϵ
    ] / 2
end

function construct_C(params)
    m_Re = reshape(params[4:12], (3, 3))
    m_Im = reshape(params[13:21], (3, 3))
    M = m_Re + im * m_Im
    return M * M'
end


# Define the objective function for optimization
function kossak_obj(params, ρ, t)
    H = construct_H(params)
    C = construct_C(params)

    obj = 0.0
    for i in 3:length(ρ)
        ρ1 = ρ[i]
        ρ2 = ρ[i - 2] + (t[i] - t[i - 1]) * (Dc(ρ[i], t[i], H, C) - 4 * Dc(ρ[i - 1], t[i - 1], H, C) + Dc(ρ[i - 2], t[i - 2], H, C)) / 3
        obj += infidelity_norm(ρ1,ρ2)
    end
    return obj
end

kossak_obj (generic function with 1 method)

In [15]:
# Define initial parameters
initial_params = [0.0, 0.0, 0.0,  # H parameters: ϵ, h_Re, h_Im
                  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,  # m_Re parameters
                  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]; # m_Im parameters

#initial_params .+= 1;

21-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [16]:
# Define the objective function wrapper
function objective(params)

    objGEXY = kossak_obj(params, ρᵍ, tᵍ) + kossak_obj(params, ρᵉ, tᵉ) + kossak_obj(params, ρˣ, tˣ) + kossak_obj(params, ρʸ, tʸ)

    return objGEXY
end

objective (generic function with 1 method)

In [17]:
# Define the density matrix evolution and time points

file_name = "../DATA/ALL_GAMMAS_B4_D10.h5"

#γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

γ = "0.25133"
γᶠ = parse(ComplexF64, γ)

tᵍ, ρᵍ = read_timeevolution(file_name, "B1", γ)
tᵉ, ρᵉ = read_timeevolution(file_name, "B2", γ)
tˣ, ρˣ = read_timeevolution(file_name, "B3", γ)
tʸ, ρʸ = read_timeevolution(file_name, "B4", γ);

In [18]:
# Perform the optimization
result = optimize(objective, initial_params, BFGS())

# Extract optimized parameters
optimized_params = result.minimizer

println("Optimized parameters: ", optimized_params)

InexactError: InexactError: Float64(32.695882843330764 + 2.9420374492301747e-16im)

In [8]:
# Substitute the optimized parameters back into H and C
optimized_H = construct_H(optimized_params)
optimized_C = construct_C(optimized_params)

println("Optimized H: ", optimized_H)
println("Optimized C: ", optimized_C)

Optimized H: ComplexF64[-37.01711919481118 + 0.0im 0.0005283032515956767 + 0.0047154387231420295im; 0.0005283032515956767 - 0.0047154387231420295im 37.01711919481118 + 0.0im]
Optimized C: ComplexF64[-1.4970537606257934 + 0.0im 0.0 - 0.7544809023334264im 0.0 + 0.0010658021670910036im; 0.0 + 0.7544809023334264im -0.016876039520281827 + 0.0im -0.0 + 0.007840533162501363im; 0.0 - 0.0010658021670910036im -0.0 - 0.007840533162501363im 0.04393659463284538 + 0.0im]


In [9]:
optimized_H

2×2 Matrix{ComplexF64}:
    -37.0171+0.0im         0.000528303+0.00471544im
 0.000528303-0.00471544im      37.0171+0.0im

In [10]:
optimized_C

3×3 Matrix{ComplexF64}:
 -1.49705+0.0im              0.0-0.754481im          0.0+0.0010658im
      0.0+0.754481im   -0.016876+0.0im              -0.0+0.00784053im
      0.0-0.0010658im       -0.0-0.00784053im  0.0439366+0.0im